In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES
from torchHMM.model.discretized_flow_HMM import FlowHMM

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [4]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)

In [5]:
session_train, sessions_test = cs.sessions[:-100000], cs.sessions[-100000:]


In [6]:
item_ids = np.unique([e.item_id for s in session_train for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list] for s in session_train]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target =  [[item_ids[e.item_id] for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target = [l for l in target if len(l)]
lt = np.array([len(y) for y in target])
target = np.concatenate(target).reshape(-1, 1)

In [7]:
len(item_ids.keys())

237227

In [8]:
chmm = CategoricalHMM(5, n_iter=10, verbose=True)


In [9]:
%time chmm.fit(X, l)

         1   -48768788.6538             +nan
         2   -44360976.0914    +4407812.5624
         3   -43987280.8218     +373695.2697
         4   -43620655.8453     +366624.9765
         5   -43315283.3347     +305372.5106
         6   -43092289.4902     +222993.8446
         7   -42932328.0870     +159961.4032
         8   -42812691.7868     +119636.3002
         9   -42720837.5345      +91854.2523


CPU times: user 10min 3s, sys: 155 ms, total: 10min 3s
Wall time: 10min 4s


        10   -42649590.1843      +71247.3502


CategoricalHMM(n_components=5,
               random_state=RandomState(MT19937) at 0x7EFFA8960A40,
               verbose=True)

In [10]:
print(chmm.score(target, lt))

-8704225.434043149


In [11]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_20_10_5_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

FileNotFoundError: [Errno 2] No such file or directory: 'clickstream_experiment/data/preprocessed_data/vectors_train_20_10_5_cleaned.kv.vectors.npy'

In [12]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

NameError: name 'vecs' is not defined

In [13]:
Xs_c = [[get_vec(e) for e in s.event_list] for s in session_train]
l_c = np.array([len(y) for y in X])

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c =  [[get_vec(e) for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target_c = np.concatenate([a for as_ in target_c for a in as_], axis=0)


NameError: name 'get_vec' is not defined

In [14]:
w2vhmm = GaussianHMM(5, n_iter=10, verbose=True)

In [15]:
# %time w2vhmm.fit(X_c, l_c)

In [16]:
# w2vhmm.transmat_

In [17]:
# zeros = w2vhmm.transmat_.sum(axis=1) == 0
# w2vhmm.transmat_[zeros] = np.ones(w2vhmm.transmat_.shape)[zeros] / w2vhmm.n_components


In [18]:
# print(w2vhmm.score(target_c, lt))

In [19]:
w2v_dhmm = DiscreteHMM(n_components=5, no_nodes=64, optimizer="Adam", verbose=True, covariance_type='full')

In [20]:
%time w2v_dhmm.fit(X_c, l_c)

NameError: name 'X_c' is not defined

In [21]:
print(w2v_dhmm.score(target_c, lt))

NameError: name 'target_c' is not defined

In [22]:
w2v_fhmm = FlowHMM(n_components=5, no_nodes=512, l=100, optimizer="Adam", verbose=True)

%time w2v_fhmm.fit(X_c, l_c)

print(w2v_fhmm.score(target_c, lt))


NameError: name 'X_c' is not defined

NameError: name 'target_c' is not defined